In [37]:
import pandas as pd
import numpy as np
from slugify import slugify
pd.set_option('display.max_columns', None)
pd. set_option("display.max_rows", None)
from Levenshtein import distance

## I. Selecionar os dados: determinar quais conjuntos de dados serão utilizados e documentar os motivos de inclusão/exclusão.

In [38]:
df = pd.read_csv("../data/raw/iowa_liquor_train_test_split_sample.csv",dtype={
    'Invoice/Item Number': str,
    'Date':str,
    'Store Number': float,
    'Store Name': str,
    'Address': str,
    'City': str,
    'Zip Code': str,
    'Store Location': str,
    'County Number': float,
    'County': str,
    'Category': float,
    'Category Name': str,
    'Vendor Number': float,
    'Vendor Name': str,
    'Item Number': float,
    'Item Description': str,
    'Pack': float,
    'Bottle Volume (ml)': int,
    'State Bottle Cost': str,
    'State Bottle Retail': str,
    'Bottles Sold': int,
    'Sale (Dollars)': str,
    'Volume Sold (Liters)': float,
    'Volume Sold (Gallons)': float
} ,encoding = 'utf-8')

In [39]:
df.columns = [slugify(col, lowercase=True, separator='_') for col in df.columns]
df.columns

Index(['unnamed_0', 'invoice_item_number', 'date', 'store_number',
       'store_name', 'address', 'city', 'zip_code', 'store_location',
       'county_number', 'county', 'category', 'category_name', 'vendor_number',
       'vendor_name', 'item_number', 'item_description', 'pack',
       'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail',
       'bottles_sold', 'sale_dollars', 'volume_sold_liters',
       'volume_sold_gallons'],
      dtype='object')

df.head(1)

### Remoção das colunas as quais não iremos utilizar
#### Vamos deixar somente as colunas data, store_location, state_bottle_retail, salle_dollars, volume_sold_liters, pois vamos realizar regressão linear em dados numéricos

In [40]:
df = df.drop(columns=['unnamed_0', 'invoice_item_number','store_number','store_name','address','zip_code','county_number','category','vendor_number','vendor_name','item_number','pack','bottle_volume_ml','state_bottle_cost','bottles_sold','volume_sold_gallons','city','category_name','item_description','county'])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518216 entries, 0 to 2518215
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   date                 object 
 1   store_location       object 
 2   state_bottle_retail  object 
 3   sale_dollars         object 
 4   volume_sold_liters   float64
dtypes: float64(1), object(4)
memory usage: 96.1+ MB


In [42]:
df.head()

,date,store_location,state_bottle_retail,sale_dollars,volume_sold_liters
0,02/26/2013,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",$8.99,$8.99,0.38
1,08/22/2017,1500 North Jefferson\nIndianola 50125\n,$47.99,$287.94,1.75
2,08/28/2013,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",$14.82,$29.64,1.50
3,03/26/2013,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",$22.72,$22.72,1.00
4,08/17/2017,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",$17.25,$17.25,0.50


In [43]:
df.shape

(2518216, 5)

In [44]:
df[df['date'].isna()]

,date,store_location,state_bottle_retail,sale_dollars,volume_sold_liters


#### Procurando por mais inconsistências

In [45]:
df[df['store_location'].isna()].shape

(467, 5)

## II. Limpar dados: Corrigir, imputar ou remover valores erroneamente inseridos nos conjuntos de dados.

In [46]:
df = df.dropna()

#### Transformando a coluna data para formato date

In [47]:
df['date'] = pd.to_datetime(df['date'])

pd. set_option("display.max_rows", None)#### Padronizando valores

In [48]:
df.head()

,date,store_location,state_bottle_retail,sale_dollars,volume_sold_liters
0,2013-02-26,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",$8.99,$8.99,0.38
1,2017-08-22,1500 North Jefferson\nIndianola 50125\n,$47.99,$287.94,1.75
2,2013-08-28,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",$14.82,$29.64,1.50
3,2013-03-26,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",$22.72,$22.72,1.00
4,2017-08-17,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",$17.25,$17.25,0.50


#### Procurando por mais inconsistências

##### cidades e condados com nome parecido

In [49]:
df['state_bottle_retail'] = df['state_bottle_retail'].replace({'\$':''}, regex = True).astype(float)

In [50]:
df['sale_dollars'] = df['sale_dollars'].replace({'\$':''}, regex = True).astype(float)

In [51]:
df['volume_sold_liters'] = df['volume_sold_liters'].astype(float)

In [52]:
df.head()

,date,store_location,state_bottle_retail,sale_dollars,volume_sold_liters
0,2013-02-26,"102 W MAYNE\nBLUE GRASS 52726\n(41.509114, -90...",8.99,8.99,0.38
1,2017-08-22,1500 North Jefferson\nIndianola 50125\n,47.99,287.94,1.75
2,2013-08-28,"1250 N LAKE ST\nSTORM LAKE 50588\n(42.653594, ...",14.82,29.64,1.50
3,2013-03-26,"316 MAIN ST\nMANNING 51455\n(41.908573, -95.06...",22.72,22.72,1.00
4,2017-08-17,"218, 6th Ave #101\nDes Moines 50309\n(41.58568...",17.25,17.25,0.50


## III. Construir dados: derivar novos atributos que serão úteis. Por exemplo, derivar o IMC de alguém a partir da altura e peso.

In [53]:
df.store_location = df.store_location.str[-22:-1]

In [54]:
df.rename(columns={'store_location':'coordinates'}, inplace=True)

In [55]:
df.head()

,date,coordinates,state_bottle_retail,sale_dollars,volume_sold_liters
0,2013-02-26,"41.509114, -90.765769",8.99,8.99,0.38
1,2017-08-22,erson\nIndianola 50125,47.99,287.94,1.75
2,2013-08-28,"42.653594, -95.200758",14.82,29.64,1.50
3,2013-03-26,"41.908573, -95.065122",22.72,22.72,1.00
4,2017-08-17,"41.585684, -93.624533",17.25,17.25,0.50


In [ ]:
# coordenadas geográficas

## IV. Integrar dados: criar novos conjuntos de dados combinando dados de várias fontes.

## V.  Formatar dados: Formatar novamente os dados conforme as necessidades dos modelos.